In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
#Adding Binaryverdict column for Grouchy
df = pd.read_csv('C:/Users/munee/Desktop/SP/Training_01092023.csv')
df['binaryverdict_g'] = df['classification'].apply(lambda x: 0 if x == 'Marshal Grouchy' else 1)
df

In [ ]:
X = df[['previous_trend_duration','previous_percentage','harpoon_output_signal','harpoon_fuel','harpoon_burnout','harpoon_h9','harpoon_h21','harpoon_h14','top_gun_output_signal','top_gun_top_gun_a','top_gun_top_gun_b','top_gun_fuel','top_gun_burnout','maverick_output_signal','maverick_fuel','maverick_burnout','wind_output_signal','wind_crossover','sonar_output_signal','sonar_fuel','sonar_burnout','Storm','Storm_Actual','Storm_Impact','Storm_MA30','Storm_MA7']]
y = df['binaryverdict_g']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
X_train = np.array(X_train)

In [ ]:
#replacing NaNs and normalizing training data
column_means = np.nanmean(X_train, axis=0)
nan_indices = np.isnan(X_train)
X_train[nan_indices] = np.take(column_means, np.where(nan_indices)[1])
X_train_norm = normalize(X_train)
X_train_norm.shape

In [ ]:
#replacing NaNs and normalizing test data
X_test = np.array(X_test)
column_means = np.nanmean(X_test, axis=0)
nan_indices = np.isnan(X_test)
X_test[nan_indices] = np.take(column_means, np.where(nan_indices)[1])
X_test_norm = normalize(X_test)
X_test_norm.shape

In [ ]:
svm_model = SVC(kernel='linear', C=1.0, random_state=42, probability=True)
output = svm_model.fit(X_train_norm, y_train)

In [ ]:
y_pred = svm_model.predict(X_test_norm)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
#Creating function to plot hyperplane
def plot_svm_decision_boundary(clf, X, y):
    xx, yy = np.meshgrid(np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 100),
                         np.linspace(X[:, 1].min() - 1, X[:, 1].max() + 1, 100))

    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contour(xx, yy, Z, levels=[-1, 0, 1], linestyles=['--', '-', '--'], colors='k')

    plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100, facecolors='none', edgecolors='k')

In [ ]:
#plotting hyperplane
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 2], X[:, 0], c=y, cmap=plt.cm.Paired)
plot_svm_decision_boundary(svm_model, X, y)

plt.title('SVM Hyperplane Plot')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

In [ ]:
#confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

In [ ]:
#ROC curve
fpr, tpr, thresholds = roc_curve(y_test, svm_model.decision_function(X_test))
auc = roc_auc_score(y_test, svm_model.decision_function(X_test))

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()